# Nordrhein-Westfalen

Every federal state is represented by its own input directory and is processed into a NUTS level 2 directory containing a sub-folder for each discharge location. These folder names are derived from NUTS and reflect the CAMELS id. The NUTS level 2 code for Nordrhein-Westfalen is `DEA`.

To pre-process the data, you need to write (at least) two functions. One should extract all metadata and condense it into a single `pandas.DataFrame`. This is used to build the folder structure and derive the ids.
The second function has to take an id, as provided by the state authorities, called `provider_id` and return a `pandas.DataFrame` with the transformed data. The dataframe needs the three columns `['date', 'q' | 'w', 'flag']`.

For easier and unified output handling, the `camelsp` package contains a context object called `Bundesland`. It takes a number of names and abbreviations to identify the correct federal state and returns an object that holds helper and save functions.

The context saves files as needed and can easily be changed to save files with different strategies, ie. fill missing data with NaN, merge data into a single file, create files for each variable or pack everything together into a netcdf.

In [1]:
import pandas as pd
import geopandas as gpd
import numpy as np
from pandas.errors import ParserError
import os
from pprint import pprint
from tqdm import tqdm
from typing import Union, Dict
from glob import glob
from datetime import datetime as dt
from dateparser import parse
import warnings
from io import StringIO

from camelsp import Bundesland, Station


The context can also be instantiated as any regular Python class, ie. to load only the default input data path, that we will user later.

In [2]:
# the context also makes the input path available, if camelsp was install locally
BASE = Bundesland('NRW').input_path
BASE

'/home/camel/camelsp/input_data/Q_and_W/NRW_Nordrhein-Westfalen'

In [3]:
# the file Datenanfrage_CAMELS_2581119000100_20221107_140905.csv exists twice, remove (2)
for f in glob(f"{BASE}/Q&W/Datenanfrage_CAMELS_2581119000100_20221107_140905*"):
    print(f)

if os.path.exists(f"{BASE}/Q&W/Datenanfrage_CAMELS_2581119000100_20221107_140905 (2).csv"):
    os.remove(f"{BASE}/Q&W/Datenanfrage_CAMELS_2581119000100_20221107_140905 (2).csv")

/home/camel/camelsp/input_data/Q_and_W/NRW_Nordrhein-Westfalen/Q&W/Datenanfrage_CAMELS_2581119000100_20221107_140905.csv


In [4]:
with Bundesland('NRW') as bl:
    metadata = pd.read_excel(os.path.join(bl.input_path, 'Q&W/Stammdaten_CAMELS.xlsx'))

metadata

,NAME,ORT,NULLPUNKT,STATIONIER,GEBFLAECHE,UTMZONE,KOORDX,KOORDY,KOMMENTAR,GEBIETSKEN,Gewässerkennzahl,Gewässer
0,Ahmsen,4639000000100,64.285,27.158,593.00,32U,479549.677600,5.771202e+06,Grundmessstelle des Landes (GL),4639.0,46,Werre
1,Albersloh,3259000000100,48.678,27.470,321.58,32U,412463.350631,5.748891e+06,NaN,3259.0,32,Werse
2,Altena,2766930000100,154.225,29.700,1190.00,32U,407683.711900,5.682847e+06,Talsperrenbeeinflussung ab 1968 (Biggetalsperre),276693.0,2766,Lenne
3,Altenbeken 2,2781610000200,215.958,11.990,20.50,32U,494359.426900,5.734473e+06,Grundmeßstelle des Landes (GL),278161.0,27816,Beke
4,Altenburg 1,2823900000200,82.651,62.440,958.76,32U,315309.586100,5.641695e+06,Grundmessstelle,28239.0,282,Rur
...,...,...,...,...,...,...,...,...,...,...,...,...
214,Westtuennen,2786700000100,57.572,3.970,414.90,32U,421634.000000,5.724518e+06,Grundmessstelle des Landes (GL),NaN,2786,Ahse
215,Wetter_Wengern_1,2769169000100,84.805,0.480,17.58,32U,384656.787100,5.695710e+06,Grundmeßstelle des Landes (GL),2769169.0,276916,Elbsche
216,Wettringen B70,9286291000300,41.504,6.330,175.07,32U,385480.262390,5.786270e+06,NaN,928629.0,92862,Steinfurter Aa
217,Wt-Kluserbrücke,2736510000100,142.226,49.240,337.82,32U,371494.000000,5.679856e+06,Durch mehrere Talsperren beeinflusst. Seit 01....,273651.0,2736,Wupper


### Metadata reader

Define the function that extracts / reads and eventually merges all metadata for this federal state. You can develop the function here, without using the Bundesland context and then later use the context to pass extracted metadata. The Context has a function for saving *raw* metadata, that takes a `pandas.DataFrame` and needs you to identify the id column.
Here, *raw* refers to provider metadata, that has not yet been transformed into the CAMELS-de Metadata schema.

In [5]:
# the id column will be ORT
id_column = 'ORT'

## file extract and parse

Here, we need to process the filename as the `'Ort'` is contained in the filename. Looks like the metadata header is **always** to line 32, indicating a finished header by `YTYP;`. Verify this.

In [6]:
for fname in glob(os.path.join(BASE, 'Q&W/Datenanfrage_CAMELS_*')):
    df = pd.read_csv(fname, encoding='latin1', sep=';', usecols=[0,1], nrows=32, header=None)
    if df.iloc[31, 0] != 'YTYP':
        print(fname)
        

That's will make our lifes way easier. Now go for all:

In [7]:
# get all file names
filelist = sorted(glob(os.path.join(BASE, 'Q&W/Datenanfrage_CAMELS_*')))

# container for meta-header and dataframes
meta = []
data = []

# go for each file
for fname in tqdm(filelist):
    # open
    with open(fname, 'rb') as f:
        txt = f.read().decode('latin1')
    
    # split header
    header = txt.splitlines()[:32]
    
    # build the meta by hand
    tups = [l.split(';') for l in header[:-1]]
    meta_dict = {t[0]: t[1] for t in tups}

    # check the parameter
    if meta_dict['Parameter'] == 'Wasserstand':
        variable = 'w'
    elif meta_dict['Parameter'] == 'Abfluss':
        variable = 'q'
    else:
        raise RuntimeError(f"Unknown Parameter: {meta_dict['Parameter']}")

    meta.append(meta_dict)
    
    # now get the body
    body = txt.splitlines()[32:]

    # now this stupid check
    second_header = [i for i, l in enumerate(body) if l.startswith('Station')]
    if len(second_header) > 0:
        # THERE IS A SECOND HEADER IN THE FILE !!!! come on!
        body = body[:second_header[0]]
    
    # write to buffer
    buffer = StringIO('\n'.join(body))
    buffer.seek(0)
    
    # read from memory
    df_data = pd.read_csv(buffer, sep=';', usecols=[0,1], skiprows=32, decimal=',', header=None, na_values='LUECKE', 
                          parse_dates=[0], date_format='%d.%m.%Y %H:%M:%S')
    
    df_data.columns = ['date', variable]
    df_data['flag'] = np.NaN
    
    # append
    data.append(df_data)
    
    
print(f"Parsed {len(meta)} metadata headers and {len(data)} data files")

  0%|          | 0/436 [00:00<?, ?it/s]

100%|██████████| 436/436 [00:41<00:00, 10.58it/s]

Parsed 436 metadata headers and 436 data files


That was really stupid. Ok. Check the metadata from the data files:

In [8]:
# create dataframe from meta, drop empty columns
extra = pd.DataFrame(meta).drop(['Gewässer', 'GUELTBIS', 'GUELTVON', 'KOMMENTAR', 'PARMERKMAL', 'XEINHEIT'], axis=1)
extra

,Station,Stationsnummer,Unterbezeichnung,Einzugsgebiet,Pegelnullpunkt,Parameter,Einheit,Aussage,Lebenslauf,Zeitangabe,...,MESSGENAU,NWGRENZE,PUBLIZIERT,QUELLE,REIHENART,VERSION,X,XDISTANZ,XFAKTOR,Y
0,Feudingen,2581119000100,,"25,40 km²","388,738 mNHN (aktuell)",Wasserstand,cm,Mittelwert,"MITTEL('2581119000100.wk3',d)",Linke Seite des Zeitintervalls mit Intervallwert,...,0.0000,0.0000,True,P,Z,0,452579,T,1,5643232
1,Feudingen,2581119000100,,"25,40 km²","388,738 mNHN (aktuell)",Abfluss,m³/s,Mittelwert,"MITTEL('2581119000100.qk0',d)",Linke Seite des Zeitintervalls mit Intervallwert,...,0.0000,0.0000,True,P,Z,0,452579,T,1,5643232
2,Lahnhof 1,2721221000100,,"0,14 km²","546,221 mNHN (aktuell)",Wasserstand,mm,Mittelwert,"MITTEL('2721221000100.wk3',d)",Linke Seite des Zeitintervalls mit Intervallwert,...,0.0000,0.0000,True,P,Z,0,447530,T,1,5637074
3,Lahnhof 1,2721221000100,,"0,14 km²","546,221 mNHN (aktuell)",Abfluss,l/s,Mittelwert,"MITTEL('2721221000100.qk0',d)",Linke Seite des Zeitintervalls mit Intervallwert,...,0.0000,0.0000,True,P,Z,0,447530,T,1,5637074
4,Helgersdorf,2721230000100,,"0,36 km²","399,194 mNHN (aktuell)",Wasserstand,mm,Mittelwert,"MITTEL('2721230000100.wk3',d)",Linke Seite des Zeitintervalls mit Intervallwert,...,0.0000,0.0000,True,P,Z,0,442478,T,1,5634356
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
431,Legden Bhf,9286410000100,,"41,43 km²","59,257 mNHN (aktuell)",Abfluss,m³/s,Mittelwert,"MITTEL('9286410000100.qk0',d)",Linke Seite des Zeitintervalls mit Intervallwert,...,0.0000,0.0000,True,P,Z,0,368458,T,1,5767205
432,Heek,9286453000200,,"122,90 km²","47,691 mNHN (aktuell)",Wasserstand,cm,Mittelwert,"MITTEL('9286453000200.wk0',d)",Linke Seite des Zeitintervalls mit Intervallwert,...,0.0000,0.0000,True,F,Z,0,371081,T,1,5775722
433,Heek,9286453000200,,"122,90 km²","47,691 mNHN (aktuell)",Abfluss,m³/s,Mittelwert,"MITTEL('9286453000200.qk0',d)",Linke Seite des Zeitintervalls mit Intervallwert,...,0.0000,0.0000,True,P,Z,0,371081,T,1,5775722
434,Gronau,9286455000200,,"183,17 km²","33,886 mNHN (aktuell)",Wasserstand,cm,Mittelwert,"MITTEL('9286455000200.wk0',d)",Linke Seite des Zeitintervalls mit Intervallwert,...,0.0000,0.0000,True,F,Z,0,365059,T,1,5787118


Metadata `extra` has different entries for Q and W at the same station. We have to merge these two metadata entries, so that we have one metadata entry for each `Stationsnummer`.

In [9]:
#extra[extra['Stationsnummer']=='2721459000100']

print(f"Two entries for ID '2721459000100':\n{extra[extra['Stationsnummer'] == '2721459000100']}")

# all IDs have two entries -> Q and W
print(f"\nNumber of duplicated IDs: {extra['Stationsnummer'].duplicated().sum()}")

Two entries for ID '2721459000100':
     Station Stationsnummer Unterbezeichnung Einzugsgebiet  \
12  Kreuztal  2721459000100                      63,40 km²   
13  Kreuztal  2721459000100                      63,40 km²   

             Pegelnullpunkt    Parameter Einheit     Aussage  \
12  273,894 mNHN (aktuell)  Wasserstand      cm  Mittelwert   
13  273,894 mNHN (aktuell)      Abfluss    m³/s  Mittelwert   

                        Lebenslauf  \
12   MITTEL('2721459000100.wk3',d)   
13   MITTEL('2721459000100.qk0',d)   

                                          Zeitangabe  ... MESSGENAU NWGRENZE  \
12  Linke Seite des Zeitintervalls mit Intervallwert  ...    0.0000   0.0000   
13  Linke Seite des Zeitintervalls mit Intervallwert  ...    0.0000   0.0000   

   PUBLIZIERT QUELLE REIHENART VERSION       X XDISTANZ XFAKTOR        Y  
12       True      P         Z       0  429430        T       1  5645719  
13       True      P         Z       0  429430        T       1  5645719  

[2

In [10]:
# get column names where Q and W data differ (at least once)
all_ids = extra['Stationsnummer'].unique()

all_different_cols = []

for id in all_ids:
    dat = extra[extra['Stationsnummer'] == id]
    if len(dat) < 2:
        continue
    different_columns = dat.iloc[0] != dat.iloc[1]
    different_colnames = different_columns[different_columns].index.tolist()
    
    for colname in different_colnames:
        if colname in all_different_cols:
            continue
        else:
            all_different_cols.append(colname)

all_different_cols

['Parameter',
 'Einheit',
 'Lebenslauf',
 'FTOLERANZ',
 'Unterbezeichnung',
 'PUBLIZIERT',
 'QUELLE',
 'Pegelnullpunkt',
 'HOEHE']

merge q and w metadata by adding Q_ and W_ suffixes.

In [11]:
# the following columns have different values in Q and W metadata at least once and therfor get a prefix
all_different_cols = ['Parameter', 'Einheit', 'Lebenslauf', 'FTOLERANZ', 'Pegelnullpunkt', 'QUELLE', 'PUBLIZIERT', 'Unterbezeichnung', 'HOEHE']

# get all unique ids
all_ids = extra['Stationsnummer'].unique()

# container
qw_metadata = []

# go for each id
for id in all_ids:
    if id in ['9284590000100', '4761500000100']:
        # for these two stations we only have w data
        w_meta = extra[(extra['Stationsnummer'] == id) & (extra['Parameter'] == 'Wasserstand')].to_dict(orient='records')[0]
        q_meta = {}
    else:
    # get Q metadata as dict
        q_meta = extra[(extra['Stationsnummer'] == id) & (extra['Parameter'] == 'Abfluss')].to_dict(orient='records')[0]
        # get W metadata as dict
        w_meta = extra[(extra['Stationsnummer'] == id) & (extra['Parameter'] == 'Wasserstand')].to_dict(orient='records')[0]

    q_updated_meta = {}
    w_updated_meta = {}

    # add prefix to Q keys
    for key in q_meta.keys():
        if key in all_different_cols:
            q_updated_meta[f"Q_{key}"] = q_meta[key]
        else:
            q_updated_meta[key] = q_meta[key]
    
    # add prefix to W keys
    for key in w_meta.keys():
        if key in all_different_cols:
            w_updated_meta[f"W_{key}"] = w_meta[key]
        else:
            w_updated_meta[key] = w_meta[key]

    # merge Q and W metadata
    qw_metadata.append({**q_updated_meta, **w_updated_meta})

qw_metadata = pd.DataFrame(qw_metadata)
qw_metadata


,Station,Stationsnummer,Q_Unterbezeichnung,Einzugsgebiet,Q_Pegelnullpunkt,Q_Parameter,Q_Einheit,Aussage,Q_Lebenslauf,Zeitangabe,...,Y,W_Unterbezeichnung,W_Pegelnullpunkt,W_Parameter,W_Einheit,W_Lebenslauf,W_FTOLERANZ,W_HOEHE,W_PUBLIZIERT,W_QUELLE
0,Feudingen,2581119000100,,"25,40 km²","388,738 mNHN (aktuell)",Abfluss,m³/s,Mittelwert,"MITTEL('2581119000100.qk0',d)",Linke Seite des Zeitintervalls mit Intervallwert,...,5643232,,"388,738 mNHN (aktuell)",Wasserstand,cm,"MITTEL('2581119000100.wk3',d)",0.0000,38874,True,P
1,Lahnhof 1,2721221000100,,"0,14 km²","546,221 mNHN (aktuell)",Abfluss,l/s,Mittelwert,"MITTEL('2721221000100.qk0',d)",Linke Seite des Zeitintervalls mit Intervallwert,...,5637074,,"546,221 mNHN (aktuell)",Wasserstand,mm,"MITTEL('2721221000100.wk3',d)",0.0000,54622,True,P
2,Helgersdorf,2721230000100,,"0,36 km²","399,194 mNHN (aktuell)",Abfluss,l/s,Mittelwert,"MITTEL('2721230000100.qk0',d)",Linke Seite des Zeitintervalls mit Intervallwert,...,5634356,,"399,194 mNHN (aktuell)",Wasserstand,mm,"MITTEL('2721230000100.wk3',d)",0.0000,39919,True,P
3,Nauholz,2721342000100,,"3,27 km²","375,565 mNHN (aktuell)",Abfluss,l/s,Mittelwert,"MITTEL('2721342000100.qk0',d)",Linke Seite des Zeitintervalls mit Intervallwert,...,5640044,,"375,565 mNHN (aktuell)",Wasserstand,cm,"MITTEL('2721342000100.wk4',d)",0.0000,37557,True,P
4,Weidenau,2721390000100,,"134,00 km²","240,473 mNHN (aktuell)",Abfluss,m³/s,Mittelwert,"MITTEL('2721390000100.qk0',d)",Linke Seite des Zeitintervalls mit Intervallwert,...,5638491,,"240,473 mNHN (aktuell)",Wasserstand,cm,"MITTEL('2721390000100.wk3',d)",0.0000,24047,True,P
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
214,Temmingsmühle,9286270000100,,"115,80 km²","58,449 mDHHN92 (aktuell)",Abfluss,m³/s,Mittelwert,"MITTEL('9286270000100.qk1',d)",Linke Seite des Zeitintervalls mit Intervallwert,...,5774423,,"58,449 mDHHN92 (aktuell)",Wasserstand,cm,"MITTEL('9286270000100.wk3',d)",0.0000,5845,True,P
215,Wettringen B70,9286291000300,,"175,07 km²","41,504 mDHHN92 (aktuell)",Abfluss,m³/s,Mittelwert,"MITTEL('9286291000300.qk1',d)",Linke Seite des Zeitintervalls mit Intervallwert,...,5786270,,"41,504 mDHHN92 (aktuell)",Wasserstand,cm,"MITTEL('9286291000300.wk2',d)",0.0000,4150,True,P
216,Legden Bhf,9286410000100,,"41,43 km²","59,257 mNHN (aktuell)",Abfluss,m³/s,Mittelwert,"MITTEL('9286410000100.qk0',d)",Linke Seite des Zeitintervalls mit Intervallwert,...,5767205,,"59,257 mNHN (aktuell)",Wasserstand,cm,"MITTEL('9286410000100.wk0',d)",0.0000,5926,True,F
217,Heek,9286453000200,,"122,90 km²","47,691 mNHN (aktuell)",Abfluss,m³/s,Mittelwert,"MITTEL('9286453000200.qk0',d)",Linke Seite des Zeitintervalls mit Intervallwert,...,5775722,,"47,691 mNHN (aktuell)",Wasserstand,cm,"MITTEL('9286453000200.wk0',d)",0.0000,4769,True,F


Now we have to left-join the data, as each Stationsnummer exists twice. Thus, it is only the combination of Stationsnummer and variable, that makes the data unique

In [12]:
metadata = qw_metadata.join(metadata.set_index(metadata.ORT.astype(str)), on='Stationsnummer', how='left')
metadata

,Station,Stationsnummer,Q_Unterbezeichnung,Einzugsgebiet,Q_Pegelnullpunkt,Q_Parameter,Q_Einheit,Aussage,Q_Lebenslauf,Zeitangabe,...,NULLPUNKT,STATIONIER,GEBFLAECHE,UTMZONE,KOORDX,KOORDY,KOMMENTAR,GEBIETSKEN,Gewässerkennzahl,Gewässer
0,Feudingen,2581119000100,,"25,40 km²","388,738 mNHN (aktuell)",Abfluss,m³/s,Mittelwert,"MITTEL('2581119000100.qk0',d)",Linke Seite des Zeitintervalls mit Intervallwert,...,388.738,222.32,25.40,32U,452579.384800,5.643232e+06,NaN,2581119.0,258,Lahn
1,Lahnhof 1,2721221000100,,"0,14 km²","546,221 mNHN (aktuell)",Abfluss,l/s,Mittelwert,"MITTEL('2721221000100.qk0',d)",Linke Seite des Zeitintervalls mit Intervallwert,...,546.221,0.50,0.14,32U,447530.356700,5.637074e+06,NaN,2721221.0,272122,Geiersgrundbach
2,Helgersdorf,2721230000100,,"0,36 km²","399,194 mNHN (aktuell)",Abfluss,l/s,Mittelwert,"MITTEL('2721230000100.qk0',d)",Linke Seite des Zeitintervalls mit Intervallwert,...,399.194,1.20,0.36,32U,442478.359000,5.634356e+06,NaN,272123.0,27212,Werthenbach
3,Nauholz,2721342000100,,"3,27 km²","375,565 mNHN (aktuell)",Abfluss,l/s,Mittelwert,"MITTEL('2721342000100.qk0',d)",Linke Seite des Zeitintervalls mit Intervallwert,...,375.565,2.57,3.27,32U,441769.690900,5.640044e+06,NaN,2721342.0,2721342,Nauholzbach
4,Weidenau,2721390000100,,"134,00 km²","240,473 mNHN (aktuell)",Abfluss,m³/s,Mittelwert,"MITTEL('2721390000100.qk0',d)",Linke Seite des Zeitintervalls mit Intervallwert,...,240.473,130.83,134.00,32U,431531.781800,5.638491e+06,Pegel seit 1971 durch Obernautalsperre und sei...,272139.0,272,Sieg
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
214,Temmingsmühle,9286270000100,,"115,80 km²","58,449 mDHHN92 (aktuell)",Abfluss,m³/s,Mittelwert,"MITTEL('9286270000100.qk1',d)",Linke Seite des Zeitintervalls mit Intervallwert,...,58.449,22.47,115.80,32U,387476.228461,5.774423e+06,NaN,928627.0,92862,Steinfurter Aa
215,Wettringen B70,9286291000300,,"175,07 km²","41,504 mDHHN92 (aktuell)",Abfluss,m³/s,Mittelwert,"MITTEL('9286291000300.qk1',d)",Linke Seite des Zeitintervalls mit Intervallwert,...,41.504,6.33,175.07,32U,385480.262390,5.786270e+06,NaN,928629.0,92862,Steinfurter Aa
216,Legden Bhf,9286410000100,,"41,43 km²","59,257 mNHN (aktuell)",Abfluss,m³/s,Mittelwert,"MITTEL('9286410000100.qk0',d)",Linke Seite des Zeitintervalls mit Intervallwert,...,59.257,81.73,41.43,32U,368457.541300,5.767205e+06,Beeinflussung durch Wehrsteuerungen und eine g...,NaN,92864,Dinkel
217,Heek,9286453000200,,"122,90 km²","47,691 mNHN (aktuell)",Abfluss,m³/s,Mittelwert,"MITTEL('9286453000200.qk0',d)",Linke Seite des Zeitintervalls mit Intervallwert,...,47.691,70.72,122.90,32U,371080.973200,5.775722e+06,Beeinflussung durch Wehrsteuerungen und ein Ho...,NaN,92864,Dinkel


In [13]:
id_column = 'Stationsnummer'

### Finally run

Now, the Q and W data can be extracted. The cool thing is, that all the id creation, data creation, merging and the mapping from our ids to the original ids and files is done by the context. This is helpful, as we less likely screw something up.

In [14]:
with Bundesland('NRW') as bl:
    # save the metadata
    bl.save_raw_metadata(metadata, 'Stationsnummer', overwrite=True)

    # for reference, call the nuts-mapping as table
    nuts_map = bl.nuts_table
    print(nuts_map.head())

    
    with warnings.catch_warnings(record=True) as warns:
        for m, df in tqdm(zip(meta, data), total=len(meta)):
            # check the meta
            provider_id = str(m['Stationsnummer'])
            bl.save_timeseries(df, provider_id)

        # check if there were warnings (there are warnings)
        if len(warns) > 0:
            log_path = bl.save_warnings(warns)
            print(f"There were warnings during the processing. The log can be found at: {log_path}")


    nuts_id    provider_id                              path
0  DEA10000  2581119000100  ./DEA/DEA10000/DEA10000_data.csv
1  DEA10010  2721221000100  ./DEA/DEA10010/DEA10010_data.csv
2  DEA10020  2721230000100  ./DEA/DEA10020/DEA10020_data.csv
3  DEA10030  2721342000100  ./DEA/DEA10030/DEA10030_data.csv
4  DEA10040  2721390000100  ./DEA/DEA10040/DEA10040_data.csv


  1%|          | 4/436 [00:00<00:41, 10.29it/s]

100%|██████████| 436/436 [00:32<00:00, 13.27it/s]
